# `MNIST like image capturing and processing`

## Importing required libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Code for capturing and generating images

In [ ]:
try:
    os.mkdir('Captured Images')
except:
    pass
    
webcam = cv2.VideoCapture(0)
i=1
while True:
    try:
        check,frame = webcam.read()
        frame = cv2.flip(frame,1)
        frame = cv2.rectangle(frame,(200,100),(500,400),(0,255,0),2) # defining a rectangular boundary for ROI
        cv2.imshow('Capturing',frame)
        key = cv2.waitKey(1)
        
        # Character 'c' is assigned to capture an image
        if key == ord('c'):
            # Extracting the ROI from the frame
            ROI = frame[100:400, 200:500].copy()
            
            # Generating a path for saving the image
            img_path = r'Captured Images\rgb_image_{}.jpg'.format(i) 
            
            # Saving the ROI from the frame as JPG image 
            cv2.imwrite(filename=img_path,img=ROI)
            
            # Loading the saved image for coversion
            final_img = cv2.imread(img_path,cv2.IMREAD_ANYCOLOR)
            
            # Converting the image into grayscale
            final_img = cv2.cvtColor(final_img,cv2.COLOR_BGR2GRAY)
            
            # Resizing the image using INTER_LINEAR interpolation (defalut case)
            final_img = cv2.resize(final_img,(28,28),interpolation=cv2.INTER_LINEAR)
            
            # Saving the new processed image 
            cv2.imwrite(r'Captured Images\final_grayscale_image_{}.jpg'.format(i),img=final_img)
            
            print('Image {} generated!'.format(i))
            
            # Deleting the original RGB image
            os.remove(img_path)
            cv2.destroyAllWindows()
            i+=1
            
        # Character 'q' is assigned to exit from the loop
        elif key == ord('q'):
            cv2.destroyAllWindows()
            break
            
    except(KeyboardInterrupt):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

webcam.release()

## Extracting the pixel values of all the grayscale images

In [ ]:
images = os.listdir('Captured Images')
pixel_values = list()

for image in images:
    path = r'Captured Images/{}'.format(image)
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    img = cv2.bitwise_not(img)
    temp = list()
    temp.append(25)
    for r in range(28):
        for c in range(28):
            temp.append(img[r,c])
    pixel_values.append(temp)

pixel_values = np.array(pixel_values)

## Generating a CSV file 

In [ ]:
columns = list()
columns.append('label')
for i in range(1,785):
    columns.append('pixel'+str(i))

df = pd.DataFrame(pixel_values,columns=columns)
df.to_csv('generated_data.csv',index=False)

## Visulaizing the generated dataset

In [ ]:
X = pd.read_csv('Path to generated_data.csv')
y = X['label']
X.drop(labels='label',axis=1,inplace=True)

In [ ]:
X.head()

In [ ]:
X.describe()

In [ ]:
X = np.array(X).reshape((X.shape[0],28,28))

In [ ]:
X=X/255

In [ ]:
figure,axes = plt.subplots(2,3,figsize=(10,7))
ind = 0
for i in range(2):
    for j in range(3):
        sns.heatmap(X[ind].reshape(28,28),annot=False,cmap='binary',ax=axes[i,j], cbar=False, xticklabels=False, yticklabels=False)
        ind+=1
plt.show()